### Testing Optimal Rotation Lists

The rotation list is a series of (cumulative) degree rotations an image is passed through if a matching barcode is not found. A long list increases the runtime before the image is given up on. However a short list may skip over crooked barcodes.

To run this notebook it needs to be placed in the root directory. This test runs as follows: using the same, image with an even barcode: iterate over rotations testing which rotation list is faster and better at getting all codes.

In [4]:
from libs.bcRead import bcRead
import cv2
import rawpy
from rawpy import LibRawNonFatalError, LibRawFatalError
import numpy as np

In [2]:
def openImageFile(imgPath, demosaic=rawpy.DemosaicAlgorithm.AHD):
        """ given an image path, attempts to return a numpy array image object
        """
        usr_gamma = 2.2
        gamma_value = (usr_gamma, usr_gamma)
        try:  # use rawpy to convert raw to openCV
            with rawpy.imread(imgPath) as raw:
                im = raw.postprocess(chromatic_aberration=(1, 1),
                                      demosaic_algorithm=demosaic,
                                      gamma=gamma_value,
                                      output_color=rawpy.ColorSpace.raw)

        # if it is not a raw format, just try and open it.
        except LibRawNonFatalError:
            bgr = cv2.imread(imgPath)
            im = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        except LibRawFatalError:
            raise
        return im

def rotim(img, angle):
    """ given a np array image object (img), and an angle rotates the img
        without cropping the corners.
    """
    # see: https://stackoverflow.com/questions/48479656/how-can-i-rotate-an-ndarray-image-properly
    # https://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/

    (height, width) = img.shape[:2]
    (cent_x, cent_y) = (width // 2, height // 2)
    mat = cv2.getRotationMatrix2D((cent_x, cent_y), -angle, 1.0)
    cos = np.abs(mat[0, 0])
    sin = np.abs(mat[0, 1])
    n_width = int((height * sin) + (width * cos))
    n_height = int((height * cos) + (width * sin))
    mat[0, 2] += (n_width / 2) - cent_x
    mat[1, 2] += (n_height / 2) - cent_y

    rotated_img = cv2.warpAffine(img, mat, (n_width, n_height))
    return rotated_img

In [5]:
imgPath = './exampleImages/various_images/query1.jpg'
im = openImageFile(imgPath)
import time

for i, rotationList in enumerate( [ [9,15,18], [18,15,9], [9,25,18], [9,15,9,18] ]):
    bcReader = bcRead('HTTU', 6, rotationList)
    failed = []
    startTime = time.time()
    for n in range(0, 90, 7):
        img = rotim(im,n)
        result = bcReader.decodeBC(img, return_details=True)
        if len(result) == 0:
            failed.append(n)
    runTime = time.time() - startTime
    print(f' for list {i}:')
    print(f'required: {runTime} seconds')
    print(f'{failed} degrees failed')


 for list 0:
required: 84.78585410118103 seconds
[21, 28] degrees failed
 for list 1:
required: 82.47723722457886 seconds
[21, 28] degrees failed
 for list 2:
required: 83.81493592262268 seconds
[21] degrees failed
 for list 3:
required: 97.86682343482971 seconds
[21] degrees failed


For now, the list: `[9,25,18]` appears to be the best although a much longer more iterative testing procedure may prove mildly useful.